# Script zum automatisierten Abzug von Kursdaten & Tweet-Klassifizierung zu Kryptowährung

Das folgende Script soll auf einem Linuxserver via Cronjob automatisiert alle 30 Minuten ausgeführt werden.  
Im ersten Schritt werden bei jeder Ausführung die aktuellen Kursdaten (bspw. aktueller Kurs, Marktkapitalisierung) zu den relevantesten Krypto Währungen von der Internetseite https://crypto.com/price abgezogen, aufbereitet und in eine .csv Datei abgespeichert.  
Anschließend werden über die Twitter-API Tweets zu den zuvor ausgelesenen Kryptowährungen abgezogen und mit Hilfe der Bibliothek "TextBlob" einer Sentiment Analysis unterzogen.
  
Bei jeder Ausführung des Scripts werden die neuen Informationen (Kursdaten & Tweet-Klassifizierungen) in die jeweiligen .csv Dateien angehängt, sodass eine fortlaufende Historie entsteht.  
  
Die hierbei entstehenden .csv Dateien sollen dann im Anschluss zur Analyse verwendet werden. Die Analyse findet in einem anderen Jupyter Notebook statt, welches nicht auf dem Server ausgeführt werden muss.

## 1. Setup

Zuerst müssen die notwendigen Module oder libraries importiert werden. der folgende Code ist nur für die Ausführung auf einem lokalen Rechner relevant. Damit das Script auf dem Linux Server läuft, wurden die Module manuell via pip auf dem Server installiert.

In [1]:
#Basics
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import csv

#Webcrawling
#pip install beautifulsoup4
from bs4 import BeautifulSoup
import requests

In [4]:
# Für Twitter Abzug & Klassifizierung
import tweepy
from tweepy import OAuthHandler
from tweepy import Stream
from textblob import TextBlob 

## 2. Skript zum automatisierten Abzug der Währungsinformationen

### 2.1 Webcrawling

Zu Beginn wird eine Funktion definiert, welche die aktuellen Kursdaten zu den relevantesten Kryptowährungen von der Internetseite https://crypto.com/price abzieht. 

In [5]:
# Erstellen der Funktion zum crawlen der aktuellen Kursdaten
def get_crypto(URL):
    CryptoDF=pd.DataFrame()
    soup=BeautifulSoup(requests.get(URL).text,"html.parser")
    # in der folgenden Schleife werden aus den relevanten html Elementen "table" die benötigten Informationen 
    # zum zuvor erstellen "CryptoDF" hinzugefügt
    for currency in soup.find("table" , {"class":"chakra-table css-1qpk7f7"}).find_all("tr"):
        CryptoList = []
        for c in currency.findAll(["p","span","div","td"]):
            if c.text.startswith(".css"):
                continue
            else:
                CryptoList.append(c.text)
        try:
            CryptoDF = CryptoDF.append({"Pos":CryptoList[3], "Name":CryptoList[11], "Short":CryptoList[12], "Price in $":CryptoList[14], "24h Change in %":CryptoList[17], "24h Volume in M$":CryptoList[20], "Market Cap in M$":CryptoList[21]}, ignore_index=True)
        except:
            continue 
        # zuletzt sollen nur die relevantesten 10 Währungen übernommen werden
        # Relevanz wird hier als Marktkapitalisierung definiert, wonach die auf der Webseite aufgeführten Währungen bereits sortiert sind
        CryptoDF= CryptoDF.head(10)
    return CryptoDF

Nun kann die Funktion mit dem entsprechenden Link als Attribut aufgerufen werden, um so einen aktuellen Abzug der Kursdaten zu erzeugen:

In [6]:
CryptoDF = get_crypto("https://crypto.com/price")

In [7]:
CryptoDF

,Pos,Name,Short,Price in $,24h Change in %,24h Volume in M$,Market Cap in M$
0,1,Bitcoin,BTC,"$17,530.62",+1.72%,$16.59 B,$333.94 B
1,2,Ethereum,ETH,"$1,360.44",+4.94%,$7.51 B,$163.96 B
2,3,Tether,USDT,$1.02,-0.26%,$27.95 B,$66.26 B
3,4,BNB,BNB,$286.49,+5.01%,$803.75 M,$45.02 B
4,5,USD Coin,USDC,$1.00,+0.01%,$3.43 B,$44 B
5,6,XRP,XRP,$0.361,+3.61%,$1.24 B,$18 B
6,7,Binance USD,BUSD,$1.00,-0.09%,$6.35 B,$16.34 B
7,8,Cardano,ADA,$0.3277,+11.48%,$892.07 M,$11.12 B
8,9,Dogecoin,DOGE,$0.07958,+8.09%,$734.31 M,$10.43 B
9,10,Polygon,MATIC,$0.8852,+6.57%,$441.54 M,$7.59 B


### 2.2 Methode zum Erstellen der Historie

Als Nächstes sollen die abgezogenen Kursdaten zu einer Historie hinzugefügt werden, welche so den Kursverlauf über längere Zeit widerspiegelt.

Folgende Codezeile ließt aus einer der "HistoryDF.csv" Datei das initial erstellte Dataframe ein. An diese .csv Datei werden in den folgenden Schritten die gecrawlten Daten angehängt und wieder abgespeichert. So entesteht in der "HistoryDF.csv" eine fortlaufende Historie der Währungsdaten.

In [8]:
HistoryDF= pd.read_csv("HistoryDF.csv", index_col=0)
HistoryDF

,BTC,ETH,USDT,USDC,BNB,XRP,BUSD,DOGE,ADA,MATIC,timestamp,ValueCategory
0,16845.38,1229.05,1.02,1.00,248.76,0.3494,1.00,0.07239,0.2576,0.7960,2023-01-03 13:48:15.025826,Price in $
1,11620.00,3010.00,15110.00,1630.00,296.00,927.0000,3250.00,256.00000,137.0000,154.0000,2023-01-03 13:48:15.040849,24h Volume in M$
2,-0.02,0.02,0.11,-0.02,-0.60,0.4200,-0.00,-0.61000,0.1100,0.4400,2023-01-03 13:48:15.054823,24h Change in %
3,321950.00,148900.00,66310.00,44710.00,39280.00,17360.0000,16430.00,9490.00000,8780.0000,6820.0000,2023-01-03 13:48:15.067823,Market Cap in M$
4,16845.38,1229.05,1.02,1.00,248.76,0.3494,1.00,0.07239,0.2576,0.7960,2023-01-03 13:49:09.077439,Price in $
...,...,...,...,...,...,...,...,...,...,...,...,...
1051,327410.00,154720.00,66280.00,43930.00,41960.00,17200.0000,16430.00,9520.00000,9910.0000,7020.0000,2023-01-08 18:00:04.508626,Market Cap in M$
1052,17161.82,1286.50,1.02,1.00,270.84,0.3443,1.00,0.07301,0.2960,0.8227,2023-01-08 18:30:03.748762,Price in $
1053,8360.00,2870.00,12200.00,1550.00,357.00,435.0000,2470.00,178.00000,322.0000,151.0000,2023-01-08 18:30:03.755303,24h Volume in M$
1054,0.13,0.47,0.31,0.01,1.94,-0.7500,0.12,0.11000,5.8600,0.4400,2023-01-08 18:30:03.760897,24h Change in %


Im nächsten Schritt wird eine Funktion definiert, welche zu einem im Funktionsaufruf anzugebenden Attribut aus dem obigen "CryptoDF" (bspw. "Price in $") eine Zeile mit Zeitstempel erstellt.  
Diese Zeile kann dann an das HistoryDF angehängt werden.

In [9]:
# Funktion zum Erstellen eines Abzuges eines beliebigen Wertes mit aktueller Systemzeit 
def create_snapshot(attribute):
    TransposedDF=CryptoDF[["Short",attribute]].transpose().reset_index().rename(columns={'index':'var'})
    TransposedDF.columns = TransposedDF.iloc[0]
    TransposedDF = TransposedDF.drop(TransposedDF.index[0])
    TransposedDF = TransposedDF.rename(columns={"Short": "ValueCategory"})
    TransposedDF["timestamp"] = datetime.now()
    return TransposedDF

Hier wird die Funktion nur beispielhaft aufgerufen, in der späteren Anwendung findet zuvor eine Bereinigung des CryptoDF statt, weswegen das im HistoryDF zu sehende Format etwas von dem Format aus der folgenden Codezeile abweicht.

In [10]:
# Beispiel Nutzung der create_snapshot Funktion zum Abzug des Wertes "Price"
create_snapshot("Price in $")

,ValueCategory,BTC,ETH,USDT,BNB,USDC,XRP,BUSD,ADA,DOGE,MATIC,timestamp
1,Price in $,"$17,530.62","$1,360.44",$1.02,$286.49,$1.00,$0.361,$1.00,$0.3277,$0.07958,$0.8852,2023-01-09 17:05:20.284652


### 2.3 Anwenden der Methode, Erstellen der Historie

Nun werden die zuvor definierten Funktionen angewendet.  
Zuerst wird die Funktion zum crawlen der aktuellen Währungsinformationen ausgeführt.

In [11]:
CryptoDF = get_crypto("https://crypto.com/price")
CryptoDF

,Pos,Name,Short,Price in $,24h Change in %,24h Volume in M$,Market Cap in M$
0,1,Bitcoin,BTC,"$17,511.05",+1.57%,$16.69 B,$333.61 B
1,2,Ethereum,ETH,"$1,361.07",+4.90%,$7.58 B,$164.05 B
2,3,Tether,USDT,$1.02,-0.28%,$28.14 B,$66.26 B
3,4,BNB,BNB,$285.69,+4.99%,$804.82 M,$44.89 B
4,5,USD Coin,USDC,$1.00,-0.01%,$3.44 B,$44 B
5,6,XRP,XRP,$0.3597,+3.44%,$1.25 B,$17.99 B
6,7,Binance USD,BUSD,$1.00,-0.18%,$6.39 B,$16.33 B
7,8,Cardano,ADA,$0.3288,+11.69%,$894.47 M,$11.14 B
8,9,Dogecoin,DOGE,$0.07938,+8.01%,$743.43 M,$10.42 B
9,10,Polygon,MATIC,$0.8839,+6.35%,$445.12 M,$7.59 B


Störende Zeichen werden an dieser Stelle bereits entfernt:

In [12]:
CryptoDF = CryptoDF.replace(['%','\$','\,'], '', regex=True)
CryptoDF

,Pos,Name,Short,Price in $,24h Change in %,24h Volume in M$,Market Cap in M$
0,1,Bitcoin,BTC,17511.05,+1.57,16.69 B,333.61 B
1,2,Ethereum,ETH,1361.07,+4.90,7.58 B,164.05 B
2,3,Tether,USDT,1.02,-0.28,28.14 B,66.26 B
3,4,BNB,BNB,285.69,+4.99,804.82 M,44.89 B
4,5,USD Coin,USDC,1.00,-0.01,3.44 B,44 B
5,6,XRP,XRP,0.3597,+3.44,1.25 B,17.99 B
6,7,Binance USD,BUSD,1.00,-0.18,6.39 B,16.33 B
7,8,Cardano,ADA,0.3288,+11.69,894.47 M,11.14 B
8,9,Dogecoin,DOGE,0.07938,+8.01,743.43 M,10.42 B
9,10,Polygon,MATIC,0.8839,+6.35,445.12 M,7.59 B


Um die Spalten "24h Volume" und "Market Cap" in einen Zahlenwert umzuwandeln, müssen die Buchstaben "B" (Billion/Milliarden) und "M" (Million/Millionen) umgewandelt werden.
Das wird in der folgenden Funktion umgesetzt. Zahlen mit "B" werden mit 1000 multipliziert und Zahlen mit M so wie sie sind wieder ins CryptoDF eingesetzt. So werden die Werte in Millionen $ dargestellt.

In [13]:
# B/Milliarden wird mit 1000 multipliziert und M/Millionen mit 1, sodass der Wert am Ende in Millionen angegeben ist
def transformvalues(columnname):
    list = []
    for val in CryptoDF[columnname]:
        split = val.split(' ')
        parsed_value = float(split[0])
        if split[1] == 'B':
            list.append(int(parsed_value * 1000))
        elif split[1] == 'M':
            list.append(int(parsed_value))
        else:
                print('error')            
    CryptoDF[columnname] = list

In [14]:
transformvalues("Market Cap in M$")
transformvalues("24h Volume in M$")

In [15]:
CryptoDF

,Pos,Name,Short,Price in $,24h Change in %,24h Volume in M$,Market Cap in M$
0,1,Bitcoin,BTC,17511.05,+1.57,16690,333610
1,2,Ethereum,ETH,1361.07,+4.90,7580,164050
2,3,Tether,USDT,1.02,-0.28,28140,66260
3,4,BNB,BNB,285.69,+4.99,804,44890
4,5,USD Coin,USDC,1.00,-0.01,3440,44000
5,6,XRP,XRP,0.3597,+3.44,1250,17990
6,7,Binance USD,BUSD,1.00,-0.18,6390,16329
7,8,Cardano,ADA,0.3288,+11.69,894,11140
8,9,Dogecoin,DOGE,0.07938,+8.01,743,10420
9,10,Polygon,MATIC,0.8839,+6.35,445,7590


Zum Schluss wird nun die zuvor definierte "create_snapshot" Funktion für die relevanten Spalten aufgerufen und die dadurch erzeugten Zeilen ans HistoryDF angefügt.  
Danach wird das erweiterte HistoryDF wieder als .csv abgespeichert, sodass es im nächsten Durchgang wieder eingelesen und erweitert werden kann.

In [16]:
HistoryDF = HistoryDF.append(create_snapshot("Price in $"),ignore_index=True)
HistoryDF = HistoryDF.append(create_snapshot("24h Volume in M$"),ignore_index=True)
HistoryDF = HistoryDF.append(create_snapshot("24h Change in %"),ignore_index=True)
HistoryDF = HistoryDF.append(create_snapshot("Market Cap in M$"),ignore_index=True)
HistoryDF.to_csv("HistoryDF.csv")
HistoryDF

,BTC,ETH,USDT,USDC,BNB,XRP,BUSD,DOGE,ADA,MATIC,timestamp,ValueCategory
0,16845.38,1229.05,1.02,1.0,248.76,0.3494,1.0,0.07239,0.2576,0.796,2023-01-03 13:48:15.025826,Price in $
1,11620.0,3010.0,15110.0,1630.0,296.0,927.0,3250.0,256.0,137.0,154.0,2023-01-03 13:48:15.040849,24h Volume in M$
2,-0.02,0.02,0.11,-0.02,-0.6,0.42,-0.0,-0.61,0.11,0.44,2023-01-03 13:48:15.054823,24h Change in %
3,321950.0,148900.0,66310.0,44710.0,39280.0,17360.0,16430.0,9490.0,8780.0,6820.0,2023-01-03 13:48:15.067823,Market Cap in M$
4,16845.38,1229.05,1.02,1.0,248.76,0.3494,1.0,0.07239,0.2576,0.796,2023-01-03 13:49:09.077439,Price in $
...,...,...,...,...,...,...,...,...,...,...,...,...
1055,327150.0,155280.0,66460.0,43920.0,42590.0,17220.0,16420.0,9570.0,10030.0,7060.0,2023-01-08 18:30:03.766757,Market Cap in M$
1056,17511.05,1361.07,1.02,1.00,285.69,0.3597,1.00,0.07938,0.3288,0.8839,2023-01-09 17:16:08.192325,Price in $
1057,16690,7580,28140,3440,804,1250,6390,743,894,445,2023-01-09 17:16:08.211323,24h Volume in M$
1058,+1.57,+4.90,-0.28,-0.01,+4.99,+3.44,-0.18,+8.01,+11.69,+6.35,2023-01-09 17:16:08.224360,24h Change in %


## 3. Script zum automatisierten Abzug von Tweets

### 3.1 Methode zum Abrufen der Twitter API

Zu Beginn wird die bereits erstellte CSV-Datei eingelesen. Diese wurde initial mit allen benötigten Spalten angelegt und hier eingelesen. In den folgenden Codezeilen wird der Inhalt erstellt, welcher am Ende des Notebooks wieder in die CSV-Datei geschrieben wird.

In [285]:
TwitterDF= pd.read_csv("TwitterDF.csv", index_col=0)

Nun wird das BearerToken festgelegt, welches benötigt wird, um die Twitter API v2 zu nutzen.  
Hierfür wurde ein Account auf https://developer.twitter.com mit Essential Access angelegt.

In [286]:
BearerToken = 'AAAAAAAAAAAAAAAAAAAAANUSkAEAAAAAPuizqhzPfOPmzsUISkQoxq1zi%2BI%3DuAlv2ZACzeHIINh7xzXyqDgVZDvonTsN667e5vsoohZNDlXOri'

Für den Abzug von Tweets muss zunächst eine Client-Verbindung unter Verwendung des Bearer Token hergestellt werden.

In [287]:
# create your client 
client = tweepy.Client(BearerToken)

Nachfolgend wird eine Methode zum Crawlen von Tweets definiert. Dieser muss ein Suchkriterium (hier *crypto* ) mitgegeben werden, welches in den Tweets enthalten sein muss. Zudem ist eine Start- und Endzeit der Suche mitzugeben sowie die Anzahl maximaler Resultate. Der Essential Access erlaubt eine maximale Anzahl von 100 Tweets pro Request, eine Anzahl von 10 ist das Minimum.  
In der Methode wird die Query durch das Suchkriterium, der Sprache Englisch und der Deaktivierung von Retweets erstellt. Diese sowie alle weiteren Parameter werden dann an die Methode *search_recent_tweets* des Clients übergeben. Dadurch crawlt die Methode Tweets, die den Suchkriterien entsprechen.  
Anschließend wird über die Tweets iteriert und ihre Erstellzeit *created_at* sowie den Inhalt des Tweets *text* in ein Dictionary geschrieben. Diese wird um die Information des Suchkriteriums *crypto* erweitert.  
Die Methode gibt das Dictionary als Rückgabewert zurück.

In [288]:
def getTweets(crypto, start_time, end_time, max_results):
    query = crypto + ' lang:en -is:retweet'
    tweets = client.search_recent_tweets(query=query,
                                     start_time=start_time,
                                     end_time=end_time,
                                     tweet_fields = ["created_at", "text", "source"],
                                     max_results = max_results
                                    )      
    tweet_info_ls = []
    # iterate over each tweet and corresponding details
    for tweet in tweets.data:
        tweet_info = {
            'created_at': tweet.created_at,
            'text': tweet.text,
        }
        tweet_info['crypto'] = crypto
        tweet_info_ls.append(tweet_info)
    return tweet_info_ls

Aus dem zuvor erstellten CryptoDF werden die Namen aller Kryptowährungen in einer Liste gespeichert.

In [289]:
cryptoList= CryptoDF["Name"]
cryptoList

0        Bitcoin
1       Ethereum
2         Tether
3       USD Coin
4            BNB
5            XRP
6    Binance USD
7       Dogecoin
8        Cardano
9        Polygon
Name: Name, dtype: object

### 3.2 Ausführen der Methode & Abspeichern der Daten

Zur Vorbereitung des Methodenaufrufs werden die Parameter initialisiert. Als Startzeit wird die aktuelle Zeit minus 30 Minuten gesetzt (=Zeitpunkt des letzten Abzugs), Endzeit ist der aktuelle Zeitstempel. Als maximale Ergebnisanzahl wird das Maximum des Essential Access Kontos zur Twitter API v2 gesetzt (100) und ein leeres Dataframe erzeugt.

In [298]:
start_time = datetime.now() - timedelta(hours=0, minutes=30)
end_time = datetime.now()
max_results = 100
tweets_df = pd.DataFrame()

Nun wird über die Liste alle Kryptowährungen iteriert und der jeweilige Name der Kryptowährung der Methode *getTweets* als Parameter mitgegeben. Somit ist der jeweilige Name der Kryptowährung der Suchbegriff im Abzug der Tweets. Das Ergebnis wird dem eben initialisierten Dataframe *tweets_df* angehängt.  
In der nächsten Iteration geschieht das gleiche mit der nächsten Kryptowährung. Somit werden für jede Kryptowährung 100 Tweets abgezogen und im Dataframe tweets_df gespeichert.

In [ ]:
for crypto in cryptoList:
    tweet_liste = getTweets(crypto, start_time, end_time, max_results=max_results)
    tweets_df = tweets_df.append(tweet_liste, ignore_index=True)

Das Dataframe wird anschließend als .csv gespeichert.

In [300]:
tweets_df.to_csv("Tweets.csv")

In [3]:
tweets_df

,created_at,text,crypto
0,2023-01-08 16:59:56+00:00,The latest Cryptoooh!! https://t.co/4xZsUP7CVw...,Bitcoin
1,2023-01-08 16:59:56+00:00,#bitcoin #hex #PulseChain #HoneyBees\nI admire...,Bitcoin
2,2023-01-08 16:59:55+00:00,Done and many thanks $BC #Bitcoin @hannywidia ...,Bitcoin
3,2023-01-08 16:59:53+00:00,@Breedlove22 @PeterZeihan @PeterZeihan can’t u...,Bitcoin
4,2023-01-08 16:59:53+00:00,@PeterMcCormack @PeterZeihan @joerogan Additio...,Bitcoin
...,...,...,...
995,2023-01-08 16:50:34+00:00,$MASQ Metrics:\n\nUSD Price: 0.184076\nETH Hol...,Polygon
996,2023-01-08 16:50:29+00:00,@TokenizerIO 🍀 greengrower crypto mining game ...,Polygon
997,2023-01-08 16:50:26+00:00,Requesting $MATIC funds from the #Stakely Fauc...,Polygon
998,2023-01-08 16:50:22+00:00,"@victalk_eth @Daren_Market is a Web3 Shopify, ...",Polygon


### 3.3 Sentiment Analysis

Ziel des nächsten Schritts ist es, die eben abgezogenen Tweets in *positiv* , *negativ* und *neutral* zu klassifizieren. Dafür wird die Python Bibliothek "TextBlob" genutzt. (https://textblob.readthedocs.io/en/dev/quickstart.html)  

Die Tweets, welche der Methode als Parameter zusammen mit dem Namen der jeweiligen Kryptowährung mitgegeben werden, werden zunächst in Textblob Objekte umgewandelt.  
Für diese kann die Property .sentiment aufgerufen werden, welche ein Tupel aus Polarity und Subjectivity zurückgibt. Beides sind float Werte, wobei Polarity in der Range [-1.0 , 1.0] angegeben wird und kleine Werte negative Äußerungen und große Werte für positive stehen. Subjectivity wird in der Range [0.0 , 1.0] angegeben, wobei 0.0 sehr objektiv und 1.0 sehr subjektiv ist.  
In diesem Projekt ist nur die Polarity von Interesse, weshalb nur dieser Wert pro Tweet in der Liste *sentiment_values* gespeichert wird.  
Anschließend wird über die Liste iteriert und Werte größer Null als positiv, Werte kleiner Null als negativ und Werte gleich Null als neutral abgespeichert.  
In einem Dictionary wird der aktuelle Zeitstempel, der Name der Kryptowährung sowie die Anzahl an positiver, negativer und neutraler Tweets gespeichert. Zudem wird der Wert *count* als die Anzahl aller untersuchten Tweets gespeichert.  
Das Dictionary ist der Rückgabewert der Methode.

In [303]:
def sentimentClassification(tweets, crypto):
    # Classify
    sentiment_objects = [TextBlob(tweet) for tweet in tweets]
    # Create a list of polarity values and tweet text
    sentiment_values = [tweet.sentiment.polarity for tweet in sentiment_objects]
    # Initialize variables, 'pos', 'neg', 'neu'.
    pos=0
    neg=0
    neu=0

    #Create a loop to classify the tweets as Positive, Negative, or Neutral.
    # Count the number of each.

    for items in sentiment_values:
        if items>0:
            pos=pos+1
        elif items<0:
            neg=neg+1
        else:
            neu=neu+1
        
    data = {
            'time': datetime.now(),
            'cypto': crypto,
            'pos': pos,
            'neg': neg,
            'neu': neu,
            'count': len(tweets)
        }

    return data

Die eben definierte Methode muss nun je Kryptowährung aufgerufen werden. Dafür wird wieder über die cryptoList iteriert und das tweets_df auf die Tweets der jeweiligen Kryptowährung eingeschränkt. Diese Tweets werden der Methode sentimentClassification zusammen mit dem Namen der Kryptowährung als Parameter mitgegeben. Das zurückgegebene Dictionary wird dem TwitterDF angehängt.  

In [304]:
for crypto in cryptoList:
    tweets = tweets_df['text'].loc[tweets_df['crypto'] == crypto]
    data = sentimentClassification(tweets, crypto)
    TwitterDF = TwitterDF.append(data, ignore_index=True)

Das TwitterDF wird in der CSV-Datei gespeichert, welche zu Beginn des Twitter-Teils im Notebook eingelesen wurde.  
Da das Notebook alle 30 Minuten ausgeführt wird, wird das TwitterDF somit alle 30 Minuten erweitert um eine Zeile je Kryptowährung mit aktuellem Zeitstempel und der Anzahl positiver, negativer und neutraler Tweets je Währung.  
Diese Informationen können in einem lokalen Notebook analyisiert werden.

In [305]:
TwitterDF.to_csv("TwitterDF.csv")

In [306]:
TwitterDF

,time,cypto,pos,neg,neu,count
0,2023-01-03 14:17:12.831583,Bitcoin,3,1,6,10
1,2023-01-03 14:17:12.851585,Ethereum,5,0,5,10
2,2023-01-03 14:17:12.871585,Tether,3,0,7,10
3,2023-01-03 14:17:12.892585,USD Coin,3,1,6,10
4,2023-01-03 14:17:12.911587,BNB,4,1,5,10
5,2023-01-03 14:17:12.927587,XRP,7,0,3,10
6,2023-01-03 14:17:12.947587,Binance USD,0,5,5,10
7,2023-01-03 14:17:12.964587,Dogecoin,5,0,5,10
8,2023-01-03 14:17:12.982587,Cardano,3,3,4,10
9,2023-01-03 14:17:13.000587,Polygon,3,2,5,10
